In [6]:
conda install -c conda-forge scikit-surprise

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\HP\anaconda3

  added / updated specs:
    - scikit-surprise


The following packages will be downloaded:

Note: you may need to restart the kernel to use updated packages.
    package                    |            build
    ---------------------------|-----------------





==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0


    conda-4.12.0               |   py39hcbf5309_0         1.0 MB  conda-forge
    python_abi-3.9             |           2_cp39           4 KB  conda-forge
    scikit-surprise-1.1.1      |   py39h5d4886f_2         538 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         1.5 MB

The following NEW packages will be INSTALLED:

  python_abi         conda-forge/win-64::python_abi-3.9-2_cp39
  scikit-surprise    conda-forge/win-64::scikit-surprise-1.1.1-py39h5d4886f_2

The following packages will be SUPERSEDED by a higher-priority channel:

  conda              pkgs/main::conda-4.12.0-py39haa95532_0 --> conda-forge::conda-4.12.0-py39hcbf5309_0




conda-4.12.0         | 1.0 MB    |            |   0% 
conda-4.12.0         | 1.0 MB    | 1          |   2% 
conda-4.12.0         | 1.0 MB    | 3          |   3% 
conda-4.12.0         | 1.0 MB    | 6          |   6% 
conda-4.12.0         | 1.0 MB    | 7          | 

  latest version: 22.11.0

Please update conda by running

    $ conda update -n base -c defaults conda




scikit-surprise-1.1. | 538 KB    | 8          |   9% 
scikit-surprise-1.1. | 538 KB    | #1         |  12% 
scikit-surprise-1.1. | 538 KB    | #4         |  15% 
scikit-surprise-1.1. | 538 KB    | #7         |  18% 
scikit-surprise-1.1. | 538 KB    | ##3        |  24% 
scikit-surprise-1.1. | 538 KB    | ####1      |  42% 
scikit-surprise-1.1. | 538 KB    | #####9     |  60% 
scikit-surprise-1.1. | 538 KB    | ########   |  80% 
scikit-surprise-1.1. | 538 KB    | ########## | 100% 
scikit-surprise-1.1. | 538 KB    | ########## | 100% 

python_abi-3.9       | 4 KB      |            |   0% 
python_abi-3.9       | 4 KB      | ########## | 100% 
python_abi-3.9       | 4 KB      | ########## | 100% 
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


In [ ]:
conda update --all

In [7]:




from sklearn import model_selection
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, mean_squared_error, mean_absolute_error
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler

from surprise.model_selection import cross_validate
from surprise import Dataset, Reader, accuracy, KNNBasic, SVD, NMF, SlopeOne, CoClustering, BaselineOnly

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

pd.set_option('display.max_rows', 1000)



In [ ]:
econ = pd.read_csv('ecommercedata.csv')

In [ ]:
econ = econ.drop('Unnamed: 0',axis=1)

In [ ]:
econ['date'] = econ['event_time'].apply(lambda x: x.split()[0])
econ['date']=pd.to_datetime(econ['date'],format='%Y-%m-%d')
econ = econ[econ['date']<pd.Timestamp('2019-10-6 00:00:00')]
econ.to_csv('initialdf.csv')
econ

In [ ]:
econ['user_score'] = econ['event_type'].map({'view':1,'cart':10,'purchase':50})
econ['user_purchase'] = econ['event_type'].apply(lambda x: 1 if x=='purchase' else 0)
econ['price_category'] = 0
for i in econ['category_code'].unique():
    econ.loc[econ['category_code']==i,'price_category'] = pd.qcut(x=econ['price'][econ['category_code']==i],q=5,labels=[1,2,3,4,5])
econ   

In [ ]:
econ.to_csv('initialdf1.csv')

In [ ]:
for i in econ['category_code'].unique():
    econ.loc[econ['category_code']==i,'price_category'] = pd.qcut(x=econ['price'][econ['category_code']==i],q=5,labels=[1,2,3,4,5])

In [ ]:
groups = econ.groupby(['user_id','product_id'])['user_score','user_purchase'].sum().reset_index()
groups['user_purchase'] = groups['user_purchase'].apply(lambda x: 1 if x>1 else x)
groups['user_score'] = groups['user_score'].apply(lambda x: 100 if x>100 else x)

In [ ]:
std = MinMaxScaler(feature_range=(0.025, 1))
std.fit(group['user_score'].values.reshape(-1,1))
group['interaction_score'] = std.transform(group['user_score'].values.reshape(-1,1))

In [ ]:
groups = groups.merge(df[['product_id','category_code','brand','price','price_category']].drop_duplicates('product_id'),on=['product_id'])

In [ ]:
X1_train, X1_valid = model_selection.train_test_split(X_train_val,test_size=0.16, random_state=42)

In [ ]:
X1_train_val, X1_test = model_selection.train_test_split(groups,test_size=0.2, random_state=42)

In [ ]:
X_valid2 = X_valid.copy()
X_valid3 = X_valid.copy()
X_train_val2 = X_train_val.copy()

In [ ]:
products_cat = X1_train[['product_id','price_category','category_code']].drop_duplicates('product_id')
products_cat = products_cat.sort_values(by='product_id')
products_cat

In [ ]:
products_cat_matrix = np.reciprocal(euclidean_distances(np.array(product_cat['price_category']).reshape(-1,1))+1)

In [ ]:
euclideann_matrix = pd.DataFrame(product_cat_matrix,columns=product_cat['product_id'],index=product_cat['product_id'])

In [ ]:
tfidf_vectorizerr = TfidfVectorizer()
doc_terms = tfidf_vectorizerr.fit_transform(list(products_cat['category_code']))
dt_matrixes = pd.DataFrame(doc_terms.toarray().round(3), index=[i for i in products_cat['product_id']], columns=tfidf_vectorizerr.get_feature_names())

In [ ]:
cos_similar_matrixx = pd.DataFrame(cosine_similarity(dt_matrix.values),columns=products_cat['product_id'],index=products_cat['product_id'])

In [ ]:
similar_matrixx = cos_similar_matrixx.multiply(euclideann_matrix)

In [ ]:
X1_train_matrix = pd.pivot_table(X1_train,values='user_score',index='user_id',columns='product_id')
X1_train_matrix = X1_train_matrix.fillna(0)

In [ ]:
content_matrixx = X1_train_matrix.dot(similar_matrixx)
std = MinMaxScaler(feature_range=(0, 1))
std.fit(content_matrixx.values)
content_matrixx = std.transform(content_matrixx.values)
content_matrixx = pd.DataFrame(content_matrixx,columns=sorted(X1_train['product_id'].unique()),index=sorted(X1_train['user_id'].unique()))
contents_df = content_matrix.stack().reset_index()
contents_df = content_df.rename(columns={'level_0':'user_id','level_1':'product_id',0:'predicted_interaction'})
X1_valid = X1_valid.merge(contents_df,on=['user_id','product_id'])

In [ ]:
X1_valid['predicted_purchase'] = X1_valid['predicted_interaction'].apply(lambda x:1 if x>=0.5 else 0)

In [ ]:
items_price_precision = precision_score(X_valid['user_purchase'],X1_valid['predicted_purchase'])
items_price_precision

In [ ]:
items_price_recall = recall_score(X1_valid['user_purchase'],X1_valid['predicted_purchase'])
items_price_recall

In [ ]:
items_price_f1 = f1_score(X1_valid['user_purchase'],X1_valid['predicted_purchase'])
items_price_f1 

In [ ]:
content_matrixx = X1_train_matrix.dot(cos_similar_matrixx)
std = MinMaxScaler(feature_range=(0, 1))
std.fit(content_matrixx.values)
content_matrixx = std.transform(content_matrixx.values)
content_matrixx = pd.DataFrame(content_matrixx,columns=sorted(X1_train['product_id'].unique()),index=sorted(X1_train['user_id'].unique()))
contents_df = content_matrixx.stack().reset_index()
contents_df = contents_df.rename(columns={'level_0':'user_id','level_1':'product_id',0:'predicted_interaction'})
X1_valid1 = X1_valid1.merge(contents_df,on=['user_id','product_id'])

In [ ]:
X1_valid1['predicted_purchase'] = X1_valid1['predicted_interaction'].apply(lambda x:1 if x>=0.5 else 0)

In [ ]:
items_precision = precision_score(X_valid1['user_purchase'],X1_valid1['predicted_purchase'])
items_precision

In [ ]:
items_recall = recall_score(X1_valid1['user_purchase'],X1_valid1['predicted_purchase'])
items_recall

In [ ]:
items_f1 = f1_score(X1_valid1['user_purchase'],X1_valid1['predicted_purchase'])
items_f1

In [ ]:
products_cat = X1_train[['product_id','price_category','category_code','brand']].drop_duplicates('product_id')
products_cat = products_cat.sort_values(by='product_id')
products_cat

In [ ]:
tfidf_vectorizerr = TfidfVectorizer()
docs_term = tfidf_vectorizerr.fit_transform(list(products_cat['brand']))
dtt_matrix1 = pd.DataFrame(docs_term.toarray().round(3), index=[i for i in products_cat['product_id']], columns=tfidf_vectorizerr.get_feature_names())
dtt_matrix1 = dtt_matrix1 + 0.01

In [ ]:
cos_similar_matrixx1 = pd.DataFrame(cosine_similarity(dtt_matrix1.values),columns=products_cat['product_id'],index=products_cat['product_id'])

In [ ]:
similar_matrixx = cos_similar_matrixx.multiply(euclidean_matrix).multiply(cos_similar_matrixx1)

In [ ]:
X1_train_matrix = pd.pivot_table(X1_train,values='user_score',index='user_id',columns='product_id')
X1_train_matrix = X1_train_matrix.fillna(0)
content_matrixx = X1_train_matrix.dot(similar_matrixx)
std = MinMaxScaler(feature_range=(0, 1))
std.fit(content_matrixx.values)
content_matrixx = std.transform(content_matrixx.values)
content_matrixx = pd.DataFrame(content_matrixx,columns=sorted(X1_train['product_id'].unique()),index=sorted(X1_train['user_id'].unique()))
contents_df = content_matrixx.stack().reset_index()
contents_df = contents_df.rename(columns={'level_0':'user_id','level_1':'product_id',0:'predicted_interaction'})
X1_valid2 = X1_valid2.merge(content_df,on=['user_id','product_id'])

In [ ]:
X1_valid2['predicted_purchase'] = X1_valid2['predicted_interaction'].apply(lambda x:1 if x>=0.5 else 0)

In [ ]:
items_price_brand_precision = precision_score(X1_valid2['user_purchase'],X1_valid2['predicted_purchase'])
items_price_brand_precision

In [ ]:
items_price_brand_recall = recall_score(X1_valid2['user_purchase'],X1_valid2['predicted_purchase'])
items_price_brand_recall

In [ ]:
items_price_brand_f1 = f1_score(X1_valid2['user_purchase'],X1_valid2['predicted_purchase'])
items_price_brand_f1

In [ ]:
products_cat = X_train_val1[['product_id','price_category','category_code','brand']].drop_duplicates('product_id')
products_cat = products_cat.sort_values(by='product_id')
products_cat_matrix = np.reciprocal(euclideann_distances(np.array(products_cat['price_category']).reshape(-1,1))+1)
euclideann_matrix = pd.DataFrame(products_cat_matrix,columns=products_cat['product_id'],index=products_cat['product_id'])

tfidf_vectorizerr = TfidfVectorizer()
docs_term = tfidf_vectorizer.fit_transform(list(product_cat['category_code']))
dtt_matrix = pd.DataFrame(doc_term.toarray().round(3), index=[i for i in product_cat['product_id']], columns=tfidf_vectorizer.get_feature_names())
cos_similar_matrixx = pd.DataFrame(cosine_similarity(dtt_matrix.values),columns=products_cat['product_id'],index=products_cat['product_id'])

tfidf_vectorizerr = TfidfVectorizer()
docs_term = tfidf_vectorizerr.fit_transform(list(products_cat['brand']))
dtt_matrix1 = pd.DataFrame(docs_term.toarray().round(3), index=[i for i in products_cat['product_id']], columns=tfidf_vectorizerr.get_feature_names())
dtt_matrix1 = dt_matrix1 + 0.01
cos_similar_matrixx1 = pd.DataFrame(cosine_similarity(dtt_matrix1.values),columns=products_cat['product_id'],index=products_cat['product_id'])

similar_matrixx = cos_similar_matrixx.multiply(euclideann_matrix).multiply(cos_similar_matrixx1)
X1_train_val_matrix = pd.pivot_table(X1_train_val1,values='user_score',index='user_id',columns='product_id')
X1_train_val_matrix = X1_train_val_matrix.fillna(0)

In [ ]:
content_matrixx = X1_train_val_matrix.dot(similar_matrixx)
std = MinMaxScaler(feature_range=(0, 1))
std.fit(content_matrix.values)
content_matrixx = std.transform(content_matrixx.values)
content_matrixx = pd.DataFrame(content_matrixx,columns=sorted(X1_train_val1['product_id'].unique()),index=sorted(X1_train_val1['user_id'].unique()))
contents_df = content_matrixx.stack().reset_index()
contents_df = contents_df.rename(columns={'level_0':'user_id','level_1':'product_id',0:'predicted_interaction'})
X1_test = X1_test.merge(contents_df,on=['user_id','product_id'])
X1_test['predicted_purchase'] = X1_test['predicted_interaction'].apply(lambda x:1 if x>=0.5 else 0)

In [ ]:
model_precision = model_precision_score(X1_test['user_purchase'],X1_test['predicted_purchase'])
model_precision

In [ ]:
model_recall = model_recall_score(X1_test['user_purchase'],X1_test['predicted_purchase'])
model_recall

In [ ]:
f2 = f2_score(X1_test['user_purchase'],X1_test['predicted_purchase'])
f2

In [ ]:
X1_test.sort_values(by='interaction_score',ascending=False).head(1000)

In [ ]:
sampleusers_trainval = X1_train_val[X1_train_val['user_id']==518044530]
sampleusers_trainval.to_csv('sampleuser_trainval.csv')
sampleusers_trainval

In [ ]:
sampleusers_test = X1_test[X1_test['user_id']==518044530]
sampleusers_test.to_csv('sampleuser_test.csv')
sampleusers_test

In [ ]:
samples = contents_df[contents_df['user_id']==518044530].sort_values(by='predicted_interaction',ascending=False).merge(groups[['product_id','category_code','brand','price','price_category']].drop_duplicates('product_id'),on='product_id')
samples

In [ ]:
sampleusers_recommend = samples[samples['predicted_interaction']>=0.5].samples(10,random_state=0)
sampleusers_recommend.to_csv('sampleuser_recommend.csv')
sampleusers_recommend

In [ ]:
econ_model = ['Item-Based','Item/Price-Based','Item/Price/Brand-Based']
econ_scoring = ['f2','model_precision','model_recall']
econ_model_name = ['item','item_price','item_price_brand']
econ_model_list = []

for i in econ_model:
    for j in econ_scoring:
        econ_model_dic = {'Model': i,'Scoring':j, 'Score':eval('{0}_{1}'.format(econ_model_name[econ_model.index(i)],j))}
        econ_model_list.append(econ_model_dic)

In [ ]:
econ_model_df = pd.DataFrame(econ_model_list)
econ_model_df

In [ ]:
sns.set(style='white',font_scale=1.2)
fig = plt.figure(figsize=[12,9])
ax = sns.barplot(x='Econ_Model',y='Econ_Score',data=model_df,hue='Econ_Scoring',palette='Econ_Blues')
for p in ax.patches:
        ax.annotate(str((p.get_height()*100).round(1))+'%', (p.get_x()+0.06, p.get_height()+0.0005))
plt.ylim([0.03,0.08])      
plt.xlabel('',weight='bold',fontsize='15')
plt.ylabel('Score',weight='bold',fontsize='18')
plt.title('Performance of SV Across Content-Based Models',weight='bold',fontsize=17)
sns.despine()
plt.savefig('barplot.png',transparent=True, bbox_inches='tight')

In [ ]:
user_groups = groups.groupby('user_id')['user_purchase'].sum()
user_groups = pd.DataFrame(user_groups)
user_groups.sort_values(by='user_purchase')

In [ ]:
fig1 = plt.figure(figsize=[12,8])
ax1 = sns.distplot(user_group,bins=np.arange(22)-0.5,kde=False)
plt.yscale('log')
plt.xticks(range(0,21,1))
plt.ylabel('No. of Customers (Log Scale)',fontsize=17, weight='bold')
plt.xlabel('Total Unique Purchases', fontsize=17, weight='bold')
plt.title('Histogram of Purchases across Customers',weight='bold',fontsize=20)
sns.despine()
plt.savefig('hisplot.png',transparent=True, bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=[7,8])
ax = sns.countplot(econ.drop_duplicates(subset=['user_id','product_id','event_type'])['event_type'])
plt.yscale('log')
for p in ax.patches:
        ax.annotate(p.get_height().round(3), (p.get_x()+0.25, p.get_height()+100))
plt.xticks(ticks=[0,1,2],labels=['Unique Views','Unique Carts','Unique Purchases'])        
plt.xlabel('')   
plt.ylabel('Count (Log Scale)')
plt.title('Unique Customer Interactions',weight='bold',fontsize=16)
sns.despine()
plt.savefig('countplot.png',transparent=True, bbox_inches='tight')

In [ ]:
users_purchase = []
for i in range(1,6):
    X_sub = X1_test[X1_test['user_id'].isin(list(X1_test.groupby('user_id')['user_purchase'].sum().reset_index()[X1_test.groupby('user_id')['user_purchase'].sum().reset_index()['user_purchase']==i]['user_id']))]
    model_recall = model_recall_score(X1_sub['user_purchase'],X1_sub['predicted_purchase'])
    model_precision = model_precision_score(X1_sub['user_purchase'],X1_sub['predicted_purchase'])
    users_purchase.append({'No. of Purchase':i,'Recall':model_recall,'Precision':model_precision})

In [ ]:
users_purchase = pd.DataFrame(users_purchase)
users_purchase2 = users_purchase.stack().reset_index().rename(columns={'level_0':'No. of Purchase','level_1':'Metric', 0:'Score'})[users_purchase.stack().reset_index().rename(columns={'level_0':'No. of Purchase','level_1':'Metric', 0:'Score'})['Metric']!='No. of Purchase']
users_purchase2['No. of Purchase'] = users_purchase2['No. of Purchase'].apply(lambda x:x+1)
users_purchase2

In [ ]:
users_purchase.stack().reset_index()

In [ ]:
user_groups = X1_test.groupby('user_id')['user_purchase'].sum()
user_groups = pd.DataFrame(user_groups)
user_groups.sort_values(by='user_purchase')

In [ ]:
fig1 = plt.figure(figsize=[10,8])
ax1 = sns.distplot(user_group,bins=np.arange(7)-0.5,kde=False)
plt.yscale('log')
plt.xticks(range(0,6,1))
plt.ylabel('No. of Customers (Log Scale)',fontsize=17, weight='bold')
plt.xlabel('Total Unique Purchases', fontsize=17, weight='bold')
ax2 = plt.twinx()
sns.lineplot(x='No. of Purchase',y='Score',data=user_purchase2, hue='Metric',ax=ax2,marker='o',sort=False,palette='magma')
plt.ylim([0,1.1])
plt.ylabel('Recall/Precision Score',fontsize=17,weight='bold') 
plt.title('Purchases and Recall/Precision of Test Set',weight='bold',fontsize=20)
plt.savefig('mixplot.png',transparent=True, bbox_inches='tight')

In [ ]:
from matplotlib.colors import LogNorm
satisfaction_confusion = confusion_matrix(X1_test['user_purchase'], X1_test['predicted_purchase'])
plt.figure(dpi=120)
ax = sns.heatmap(satisfaction_confusion, cmap=plt.cm.Blues, annot=True, square=True, fmt='d',norm=LogNorm(),
xticklabels=['Did Not Purchase', 'Purchase'],
yticklabels=['Did Not Purchase', 'Purchase']);   
plt.xlabel('Prediction',weight='bold',fontsize=12)
plt.ylabel('Actual',weight='bold',fontsize=12)
plt.title('Confusion Matrix',weight='bold',fontsize=15)
plt.savefig('confusionmatrix.png',transparent=True, bbox_inches='tight')

In [ ]:
group.to_csv('groupdf.csv')